In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [128]:
#ccx train 
ccx = pd.read_csv('ccx_A_train.csv', encoding='gb2312')
ccx.head()

,lend_request_id,qm,type,result,instid,instType
0,1,2,T0,R0,B0,A0
1,3,2,T1,R0,B1,A0
2,3,2,T2,R1,B2,A1
3,3,2,T3,R2,B3,A2
4,6,2,T4,R3,B4,A3


In [129]:
#drop qm column
ccx1 = ccx.drop(['qm'],axis = 1)
ccx1.head()

,lend_request_id,type,result,instid,instType
0,1,T0,R0,B0,A0
1,3,T1,R0,B1,A0
2,3,T2,R1,B2,A1
3,3,T3,R2,B3,A2
4,6,T4,R3,B4,A3


In [130]:
#count the number of each search result and store it in a new dataframe
ccx2 = pd.DataFrame(ccx1.groupby(['lend_request_id','result']).size())
ccx2.columns=['count']
ccx2.head()

count
lend_request_id result       
1               R0          1
3               R0          1
                R1          1
                R2          2
6               R3          1

In [131]:
#transpose the result into personal level
ccx_result = pd.DataFrame(ccx2.groupby(['lend_request_id', 'result'])['count'].first().unstack(fill_value='').rename_axis(None, 1))
ccx_result.head()

,R0,R1,R10,R11,R12,R13,R14,R15,R16,R17,...,R22,R23,R24,R3,R4,R5,R6,R7,R8,R9
lend_request_id,,,,,,,,,,,,,,,,,,,,,
1,1,,,,,,,,,,...,,,,,,,,,,
3,1,1,,,,,,,,,...,,,,,,,,,,
6,,,,,,,,,,,...,,,,1,,,,,,
8,,,,,,,,,,,...,,,,1,,,,,,
9,1,,,,,,,,,,...,,,,,,,,,,


In [145]:
#export dataframe to csv file
ccx_result.to_csv(r'ccx_final.csv')

In [133]:
#consumer train 
consumer = pd.read_csv('Consumer_train.csv')
consumer.head()

,ccx_id,province,city,area,fare,goodsvalue,orderamount,paymenttime,platform,refund,discount,ordertime,deliveryNum,unitPrice,goodsName
0,1,Shandong,济宁,任城,10.0,38.0,48.0,6/4/15 09:41,淘宝分销,0.0,0.0,0000-00-00 00:00:00,1.0,38.0,书籍/杂志/报纸
1,1,Shandong,济宁,任城,10.0,38.0,48.0,6/4/15 09:41,淘宝分销,0.0,0.0,6/3/15 23:04,1.0,38.0,书籍/杂志/报纸
2,1,Jiangsu,徐州,新沂,0.0,25.0,25.0,11/28/15 20:36,淘宝,0.0,0.0,0000-00-00 00:00:00,1.0,25.0,书籍/杂志/报纸
3,1,Jiangsu,徐州,新沂,0.0,49.5,49.5,7/9/14 10:08,淘宝,0.0,0.0,0000-00-00 00:00:00,1.0,49.5,女士内衣/男士内衣/家居服
4,1,Jiangsu,徐州,新沂,0.0,12.8,12.8,10/14/16 22:16,淘宝,0.0,27.0,10/14/16 22:16,1.0,12.8,书籍/杂志/报纸


In [134]:
#Only use orderamount and goosName 
consumer2 = consumer.loc[:,('ccx_id','orderamount','goodsName')]
consumer2.head()

,ccx_id,orderamount,goodsName
0,1,48.0,书籍/杂志/报纸
1,1,48.0,书籍/杂志/报纸
2,1,25.0,书籍/杂志/报纸
3,1,49.5,女士内衣/男士内衣/家居服
4,1,12.8,书籍/杂志/报纸


In [135]:
# sum up the total amount spent in each category for every consumer
consumer_result = pd.DataFrame(consumer2.groupby(['ccx_id', 'goodsName'])['orderamount'].first().unstack(fill_value='').rename_axis(None, 1))
consumer_result.head()

,3C数码配件,OTC药品/医疗器械/计生用品,ZIPPO/瑞士军刀/眼镜,个人护理/保健/按摩器材,乐器/吉他/钢琴/配件,书籍/杂志/报纸,五金/工具,传统滋补营养品,住宅家具,保健食品/膳食营养补充食品,...,运动服/休闲服装,运动鞋new,酒类,闪存卡/U盘/存储/移动硬盘,隐形眼镜/护理液,零食/坚果/特产,音乐/影视/明星/音像,餐饮具,饰品/流行首饰/时尚饰品新,鲜花速递/花卉仿真/绿植园艺
ccx_id,,,,,,,,,,,,,,,,,,,,,
1,58,,,,,48,,,,216,...,,,,,,,,,,
2,,,,,,10.8,,,,,...,,,,,,,,,,
3,,,,,,169,,,,,...,17,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
6,,,,,,,,,,,...,,,,,,,,,,


In [136]:
consumer_result=consumer_result.convert_objects(convert_numeric=True)

/Users/Cindy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [137]:
#reclassify the 74 categories into 10 big categories
consumer_result['it_buy']=consumer_result[['闪存卡/U盘/存储/移动硬盘','平板电脑/MID','电脑硬件/显示器/电脑周边','3C数码配件',
                                           '手机号码/套餐/增值业务','网络设备/网络相关','手机']].sum(axis=1)
consumer_result['clothing_buy']=consumer_result[['女士内衣/男士内衣/家居服', '女装/女士精品', '男装', '流行男鞋', '运动服/休闲服装', '运动鞋new', '饰品/流行首饰/时尚饰品新','女鞋', '手表',  '珠宝/钻石/翡翠/黄金','服饰配件/皮带/帽子/围巾', '童鞋/婴儿鞋/亲子鞋',  '童装/婴儿装/亲子装', '箱包皮具/热销女包/男包']].sum(axis=1)
consumer_result['food_buy']=consumer_result[['粮油米面/南北干货/调味品','保健食品/膳食营养补充食品', '零食/坚果/特产','咖啡/麦片/冲饮','茶', '传统滋补营养品', '奶粉/辅食/营养品/零食', '水产肉类/新鲜蔬果/熟食', '酒类']].sum(axis=1)
consumer_result['home_buy']=consumer_result[[ '住宅家具','洗护清洁剂/卫生巾/纸/香薰', '居家日用', '家居饰品','厨房/烹饪用具', '餐饮具', '居家布艺', '商业/办公家具', '家庭/个人清洁工具', '床上用品','收纳整理', '厨房电器', '大家电',  '生活电器']].sum(axis=1)
consumer_result['ent_buy']=consumer_result[['影音电器', '音乐/影视/明星/音像',  '模玩/动漫/周边/cos/桌游', '玩具/童车/益智/积木/模型', '乐器/吉他/钢琴/配件']].sum(axis=1)
consumer_result['tools_buy']=consumer_result[['电子/电工', '基础建材', '五金/工具','办公设备/耗材/相关服务']].sum(axis=1)
consumer_result['stationery_buy']=consumer_result[['书籍/杂志/报纸', '电子词典/电纸书/文化用品', '节庆用品/礼品']].sum(axis=1)  
consumer_result['health_buy'] =consumer_result[['OTC药品/医疗器械/计生用品', '处方药',  '个人护理/保健/按摩器材', '运动/瑜伽/健身/球迷用品', '户外/登山/野营/旅行用品','自行车/骑行装备/零配件']].sum(axis=1)
consumer_result['skincare_buy']=consumer_result[['彩妆/香水/美妆工具', '美容护肤/美体/精油', '隐形眼镜/护理液']].sum(axis=1)
consumer_result['other_buy'] =consumer_result[['其他',  'ZIPPO/瑞士军刀/眼镜',  '汽车/用品/配件/改装','宠物/宠物食品及用品', '鲜花速递/花卉仿真/绿植园艺', '尿片/洗护/喂哺/推车床', '家装主材']].sum(axis=1)


In [138]:
# slice the dataframe
consumer_result2=consumer_result.iloc[:,74::]
consumer_result2.head()

,it_buy,clothing_buy,food_buy,home_buy,ent_buy,tools_buy,stationery_buy,health_buy,skincare_buy,other_buy
ccx_id,,,,,,,,,,
1,58.0,177.5,230.5,9.41,0.0,0.0,48.0,26.9,26.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0,10.8,0.0,22.9,0.0
3,0.0,17.0,0.0,39.52,0.0,0.0,169.0,0.0,0.0,0.0
4,0.0,0.0,93.7,1013.00,0.0,0.0,0.0,0.0,0.0,179.0
6,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,26.8


In [139]:
#count the number of times each consumer shopped in each category
consumer3=pd.DataFrame(consumer2.groupby(['ccx_id','goodsName']).size())
consumer3.columns=['count']
consumer3.head()

count
ccx_id goodsName           
1      3C数码配件             4
       书籍/杂志/报纸           8
       保健食品/膳食营养补充食品      1
       女士内衣/男士内衣/家居服      1
       女装/女士精品            1

In [140]:
#transpose the result into personal level
consumer_result3 = pd.DataFrame(consumer3.groupby(['ccx_id', 'goodsName'])['count'].first().unstack(fill_value='').rename_axis(None, 1))
consumer_result3.head()

,3C数码配件,OTC药品/医疗器械/计生用品,ZIPPO/瑞士军刀/眼镜,个人护理/保健/按摩器材,乐器/吉他/钢琴/配件,书籍/杂志/报纸,五金/工具,传统滋补营养品,住宅家具,保健食品/膳食营养补充食品,...,运动服/休闲服装,运动鞋new,酒类,闪存卡/U盘/存储/移动硬盘,隐形眼镜/护理液,零食/坚果/特产,音乐/影视/明星/音像,餐饮具,饰品/流行首饰/时尚饰品新,鲜花速递/花卉仿真/绿植园艺
ccx_id,,,,,,,,,,,,,,,,,,,,,
1,4,,,,,8,,,,1,...,,,,,,,,,,
2,,,,,,1,,,,,...,,,,,,,,,,
3,,,,,,4,,,,,...,1,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
6,,,,,,,,,,,...,,,,,,,,,,


In [141]:
#change data type to numerical
consumer_result3=consumer_result3.convert_objects(convert_numeric=True)

/Users/Cindy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [142]:
#reclassify the 74 categories into 10 big categories
consumer_result3['it_buy_count']=consumer_result3[['闪存卡/U盘/存储/移动硬盘','平板电脑/MID','电脑硬件/显示器/电脑周边','3C数码配件',
                                           '手机号码/套餐/增值业务','网络设备/网络相关','手机']].sum(axis=1)
consumer_result3['clothing_buy_count']=consumer_result3[['女士内衣/男士内衣/家居服', '女装/女士精品', '男装', '流行男鞋', '运动服/休闲服装', '运动鞋new', '饰品/流行首饰/时尚饰品新','女鞋', '手表',  '珠宝/钻石/翡翠/黄金','服饰配件/皮带/帽子/围巾', '童鞋/婴儿鞋/亲子鞋',  '童装/婴儿装/亲子装', '箱包皮具/热销女包/男包']].sum(axis=1)
consumer_result3['food_buy_count']=consumer_result3[['粮油米面/南北干货/调味品','保健食品/膳食营养补充食品', '零食/坚果/特产','咖啡/麦片/冲饮','茶', '传统滋补营养品', '奶粉/辅食/营养品/零食', '水产肉类/新鲜蔬果/熟食', '酒类']].sum(axis=1)
consumer_result3['home_buy_count']=consumer_result3[[ '住宅家具','洗护清洁剂/卫生巾/纸/香薰', '居家日用', '家居饰品','厨房/烹饪用具', '餐饮具', '居家布艺', '商业/办公家具', '家庭/个人清洁工具', '床上用品','收纳整理', '厨房电器', '大家电',  '生活电器']].sum(axis=1)
consumer_result3['ent_buy_count']=consumer_result3[['影音电器', '音乐/影视/明星/音像',  '模玩/动漫/周边/cos/桌游', '玩具/童车/益智/积木/模型', '乐器/吉他/钢琴/配件']].sum(axis=1)
consumer_result3['tools_buy_count']=consumer_result3[['电子/电工', '基础建材', '五金/工具','办公设备/耗材/相关服务']].sum(axis=1)
consumer_result3['stationery_buy_count']=consumer_result3[['书籍/杂志/报纸', '电子词典/电纸书/文化用品', '节庆用品/礼品']].sum(axis=1)  
consumer_result3['health_buy_count'] =consumer_result3[['OTC药品/医疗器械/计生用品', '处方药',  '个人护理/保健/按摩器材', '运动/瑜伽/健身/球迷用品', '户外/登山/野营/旅行用品','自行车/骑行装备/零配件']].sum(axis=1)
consumer_result3['skincare_buy_count']=consumer_result3[['彩妆/香水/美妆工具', '美容护肤/美体/精油', '隐形眼镜/护理液']].sum(axis=1)
consumer_result3['other_buy_count'] =consumer_result3[['其他',  'ZIPPO/瑞士军刀/眼镜',  '汽车/用品/配件/改装','宠物/宠物食品及用品', '鲜花速递/花卉仿真/绿植园艺', '尿片/洗护/喂哺/推车床', '家装主材']].sum(axis=1)
#slicing the result
consumer_result4=consumer_result3.iloc[:,74::]
consumer_result4.head()

,it_buy_count,clothing_buy_count,food_buy_count,home_buy_count,ent_buy_count,tools_buy_count,stationery_buy_count,health_buy_count,skincare_buy_count,other_buy_count
ccx_id,,,,,,,,,,
1,4.0,2.0,2.0,1.0,0.0,0.0,8.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0
4,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [143]:
#merge two dataframe 
consumer_final = pd.concat([consumer_result2,consumer_result4], axis=1, join='inner')
consumer_final.head()

,it_buy,clothing_buy,food_buy,home_buy,ent_buy,tools_buy,stationery_buy,health_buy,skincare_buy,other_buy,it_buy_count,clothing_buy_count,food_buy_count,home_buy_count,ent_buy_count,tools_buy_count,stationery_buy_count,health_buy_count,skincare_buy_count,other_buy_count
ccx_id,,,,,,,,,,,,,,,,,,,,
1,58.0,177.5,230.5,9.41,0.0,0.0,48.0,26.9,26.0,0.0,4.0,2.0,2.0,1.0,0.0,0.0,8.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.00,0.0,0.0,10.8,0.0,22.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,17.0,0.0,39.52,0.0,0.0,169.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0
4,0.0,0.0,93.7,1013.00,0.0,0.0,0.0,0.0,0.0,179.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,26.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [144]:
#export the final result into csv file
consumer_final.to_csv(r'consumer_final.csv')